In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [9]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [10]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [11]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [12]:
import bead_util as bu
# import calib_util as cal
import transfer_func_util as tf
import configuration as config

### Discharge data

In [420]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 1
decimate = 10
drive_freq = 139
fsamp = 5000
def func1_scale(i):
    bb = step_file_objs[i]
        
    freq2 = np.fft.rfftfreq(len(bb.other_data[3]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[3]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.other_data[3])*np.sqrt(2)*100)

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-10000, 100000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    bb.load_other_data()
    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     print(m1_tmp.values[2]-phi_tmp2)
    return m1_tmp.values[0]

In [421]:
func1_scale(0)

[ 3.55299124] 0.0896765355016


795.1782780584945

In [581]:
step_cal_dir = r'/data/old_trap/20211001/bead1/discharge/discahrge_20211013/'
max_file = 500

In [819]:
step_cal_files = []
for root, dirnames, filenames in os.walk(step_cal_dir):
    for filename in fnmatch.filter(filenames, '*' + config.extensions['data']):
        if '_fpga.h5' in filename:
            continue
        step_cal_files.append(os.path.join(root, filename))
step_cal_files.sort(key = bu.find_str)
num_calib_files = len(step_cal_files)
print(num_calib_files)

# Do the step calibration

step_file_objs = []
for filname in step_cal_files[320:-1]:
    try:
        df = bu.DataFile();
        df.load(filname);
        df.load_other_data()
        step_file_objs.append(df);
    except:
        print('***')

443


In [820]:
amps1 = []
for i in range(num_calib_files-321):
    print(i,'/',len(step_cal_files))
    amps1.append(func1_scale(i))

0 / 443
[ 0.83865688] 2.40751950876
1 / 443
[ 0.70023023] 2.4065770516
2 / 443
[ 1.18541548] 2.40721187804
3 / 443
[ 1.90683001] 2.4070220091
4 / 443
[ 4.73403284] 2.40698695126
5 / 443
[ 5.81740964] 2.40759450832
6 / 443
[ 4.22919449] 2.40680865909
7 / 443
[ 2.75944782] 2.40710580766
8 / 443
[ 3.49108245] 2.40705381125
9 / 443
[ 1.51771758] 2.40682420997
10 / 443
[ 5.65076309] 2.40686593949
11 / 443
[ 4.90967486] 2.40745990297
12 / 443
[ 1.99358633] 2.40647549758
13 / 443
[ 0.08044165] 2.4070012432
14 / 443
[ 4.17967922] 2.40652287879
15 / 443
[ 1.41029337] 2.40639519895
16 / 443
[ 5.78088625] 2.40722329843
17 / 443
[ 4.38435408] 2.40665630795
18 / 443
[ 1.13376259] 2.40729907669
19 / 443
[ 5.69624615] 2.4066187813
20 / 443
[ 4.38541869] 2.40679614702
21 / 443
[ 6.13153411] 2.40674431319
22 / 443
[ 0.24706033] 2.4072931741
23 / 443
[ 3.4255261] 2.40736525736
24 / 443
[ 3.07578089] 2.40682737517
25 / 443
[ 5.88540988] 2.40654617116
26 / 443
[ 1.88001327] 2.4070077156
27 / 443
[ 2.73025

In [822]:
amps = np.array(amps)
amps2 = np.array(amps2)
amps1 = np.array(amps1)
_,ax = plt.subplots(figsize=(9.5,4))
# scale = 375

# Perrinf
# ax.scatter(range(len(amps3)), amps3/scaleY, label='fit phase', alpha=0.3)
ax.scatter(range(len(amps2)), amps2/scale*1.05, label='fit phase')
ax.scatter(range(len(amps)), amps/scaleY, label='fit phase', alpha=0.4)
ax.scatter(range(len(amps1)), amps1/scale, label='fit phase', alpha=0.8)

ax.set(xlabel='dataset', ylabel='electron charges', ylim=(-1,9), xlim=(None,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator

ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [243]:
np.mean(amps3/scaleY)/4, np.mean(amps1/scaleY)/4

(1.1776399750161368, 3.3010408106832543)

In [250]:
# scale = -1*np.abs(np.mean(amps3[200:250]))/2
# scale = -1*np.abs(np.mean(amps[230:240]))/3
scale = -1*np.abs(np.mean(amps2[97:118]))/4
scaleY = np.abs(np.mean(amps[150:]))/4
print("Scales are ", scale, scaleY)
bb = step_file_objs[-1]
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[3])
voltage = np.std(xx2)*np.sqrt(2)*100
print("voltage: ", voltage)

Scales are  -98.3245603727 93.2995440303
voltage:  0.0514378232623


In [159]:
freq2 = np.fft.rfftfreq(len(bb.other_data[3]), d=1./5000)
fft_angles = np.angle(np.fft.rfft(bb.other_data[3]))
fft_angles2 = np.angle(np.fft.rfft(bb.other_data[4]))
(fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi), (fft_angles2[freq2==drive_freq]+np.pi/2)%(2*np.pi) 

(array([ 6.24211336]), array([ 3.1101641]))

In [174]:
_,ax = plt.subplots()
ax.plot(range(500),signal.filtfilt(b, a, bb.other_data[3])[2000:2500], '--')
ax.plot(range(500),signal.filtfilt(b, a, bb.other_data[4])[2000:2500], '--')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [191]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 4
decimate = 2
drive_freq = 139
electrode = 2
pos_index = 2
# phase_resp = -0.705
phase_resp = 0
def get_amp2(i):
    
    fname = r'/data/old_trap/20211001/bead1/single_electrode_20211013/Z_minus_139Hz_50Vpp/Z_50Vpp_'+str(i)+'.h5'

    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    fit_kwargs = {'A': 0.1, 'f': drive_freq, 'phi': 0.9, 
                  'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [0, 10000],
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.other_data[electrode])[::decimate]
    m1_tmp = ll.find_mle_sin(bb.other_data[electrode], drive_freq=139, fsamp=5000, bandwidth=4, noise_rms=0, decimate=10, plot=False, suppress_print=True, **fit_kwargs)
    
    freq2 = np.fft.rfftfreq(len(bb.other_data[electrode]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[electrode]))

    phi_tmp = m1_tmp.values[2]
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp%(2*np.pi), phi_tmp2, m1_tmp.values[0]*100,  np.std(xx2)*np.sqrt(2)*100)
    
    fit_kwargs = {'A': 0, 'f': 139, 'phi': phi_tmp, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'fix_f': True, 'fix_phi': False,
                  'limit_A': [-100000, 10000], 
                  'print_level': 0}

    xx2 = signal.filtfilt(b, a, bb.pos_data[pos_index])[::decimate]
    
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    return m2_tmp.values[0], m2_tmp.values[2]-phi_tmp

In [186]:
# plus_Z = np.array([get_amp2(i) for i in range(50)])
np.mean(np.abs(plus_Z[:,0])), np.mean(plus_Z[:,1]), np.std(np.abs(plus_Z[:,0])), np.std(np.abs(plus_Z[:,0]))/np.sqrt(len(plus_Z[:,0])), np.std(plus_Z[:,1])

(418.46407896921414,
 -0.98126051820833993,
 22.06206160075007,
 3.1200466729691425,
 1.1076992369157805)

In [173]:
422/368

1.1467391304347827

In [193]:
# minus_Z = np.array([get_amp2(i) for i in range(50)])
np.mean(np.abs(minus_Z[:,0])), np.mean(minus_Z[:,1]), np.std(np.abs(minus_Z[:,0])), np.std(np.abs(minus_Z[:,0]))/np.sqrt(len(minus_Z[:,0])), np.std(minus_X[:,1])

(627.69923206982025,
 -45039.678041592946,
 35.910234144313215,
 5.0784740154881138,
 0.097838730700977608)

In [882]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(50), plus_Z[:,1]%(np.pi/2))
ax.scatter(range(50), minus_Z[:,1]%(np.pi/2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [199]:
response_elec3_3 = np.mean(plus_X[:,0])
response_elec4_3 = np.mean(minus_X[:,0])
print('electrodes response - x: ', response_elec3_3, response_elec4_3, response_elec3_3/response_elec4_3)

response_elec5_3 = np.mean(plus_Y[:,0])
response_elec6_3 = np.mean(minus_Y[:,0])
print('electrodes response - y: ', response_elec5_3, response_elec6_3, response_elec5_3/response_elec6_3)

response_elec1_3 = np.mean(np.abs(plus_Z[:,0]))
response_elec2_3 = np.mean(np.abs(minus_Z[:,0]))
print('electrodes response - z: ', response_elec1_3, response_elec2_3, response_elec1_3/response_elec2_3)

electrodes response - x:  -421.449110962 368.055730209 -1.14506873924
electrodes response - y:  392.508760751 -406.3974214 -0.965824929201
electrodes response - z:  418.464078969 627.69923207 0.666663359758


In [1859]:
amps_elec2_139 = []
for i in range(20):
    print(i,'/20')
    try:
        amps_elec2_139.append(np.abs(get_amp2(i)))
    except:
        print("******")
amps_elec2_139 = np.array(amps_elec2_139)

0 /20
-4.433089210778661 [ 4.3959537] 0.0017853877265494589
1 /20
-2.8847002425531643 [ 3.94786534] 0.001787011150167217
2 /20
-5.7746064613891654 [ 4.8198017] 0.0017868101442886086
3 /20
4.619941688316747 [ 2.77083364] 0.0017882590408468957
4 /20
2.6111044550843907 [ 0.39460166] 0.0017868094781547939
5 /20
-1.3314261083994348 [ 4.64087711] 0.0017869246082824475
6 /20
5.827349316040854 [ 5.78376359] 0.0017863104329052248
7 /20
5.657360350104334 [ 0.79274317] 0.001786118697388872
8 /20
4.70766743932916 [ 0.97929161] 0.0017861860324153156
9 /20
6.226785671376437 [ 0.17947267] 0.0017865659507343423
10 /20
5.774578938111093 [ 3.05483376] 0.0017884273506574289
11 /20
-3.183667109764247 [ 3.69408964] 0.0017868146406918584
12 /20
3.004123102312697 [ 1.06174872] 0.0017876346514178465
13 /20
4.748277673933055 [ 6.15254962] 0.0017864737467121472
14 /20
-2.828988545210402 [ 3.99253185] 0.0017871281676740125
15 /20
-2.621920369169667 [ 3.52634065] 0.0017871115698397944
16 /20
0.25300685492482433 [

In [1860]:
response_elec3 = np.mean(amps_elec3)
response_elec4 = np.mean(amps_elec4)
print('electrodes response - x: ', response_elec3, response_elec4, response_elec3/response_elec4)

response_elec3_2 = np.mean(amps_elec3_139)
response_elec4_2 = np.mean(amps_elec4_139)
print('electrodes response - x: ', response_elec3_2, response_elec4_2, response_elec3_2/response_elec4_2)

response_elec5 = np.mean(amps_elec5)
response_elec6 = np.mean(amps_elec6)
print('electrodes response - y: ', response_elec5, response_elec6, response_elec5/response_elec6)

response_elec5_2 = np.mean(amps_elec5_139)
response_elec6_2 = np.mean(amps_elec6_139)
print('electrodes response - y: ', response_elec5_2, response_elec6_2, response_elec5_2/response_elec6_2)

response_elec1 = np.mean(amps_elec1)
response_elec2 = np.abs(np.mean(amps_elec2))
print('electrodes response - z: ', response_elec1, response_elec2, response_elec1/response_elec2)

response_elec1_2 = np.mean(amps_elec1_139)
response_elec2_2 = np.abs(np.mean(amps_elec2_139))
print('electrodes response - z: ', response_elec1_2, response_elec2_2, response_elec1_2/response_elec2_2)
# print('2nd harmonic: ', np.mean(amps_y_82Hz), scaleY)
# response_elec5

electrodes response - x:  2786.56266318 1884.09711644 1.47899099196
electrodes response - x:  1617.35724093 1395.04988105 1.15935441657
electrodes response - y:  2377.02864414 2759.80835902 0.861302066996
electrodes response - y:  1734.99608887 1792.16339056 0.968101512398
electrodes response - z:  7469.88596913 10588.3225531 0.705483416442
electrodes response - z:  1442.13072408 2160.0779191 0.667629029179


In [1]:
1.5*0.9681

1.45215

In [1862]:
scaleY = scale*(response_elec5_2+response_elec6_2)/(response_elec3_2+response_elec4_2)
scaleZ = scale*(response_elec1_2+response_elec2_2)/(response_elec3_2+response_elec4_2)
print(scale, scaleY, scaleZ)

-103.205784719 -120.841322956 -123.412525162


In [883]:
scale, scaleY, scaleZ = (-103.205784719, -120.841322956, -123.412525162)

In [188]:
_,ax = plt.subplots()
ax.scatter(range(20), np.abs(amps_elec4))
ax.scatter(range(20), np.abs(amps_elec5))
x = np.mean(np.abs(amps_elec4)); dx = np.std(np.abs(amps_elec4))/np.sqrt(20)
y = np.mean(np.abs(amps_elec5)); dy = np.std(np.abs(amps_elec5))/np.sqrt(20)
z = x/y; dz = np.abs(x/y)*np.sqrt((dx/x)**2+(dy/y)**2)
print(x, dx)
print(y, dy)
print(z, dz)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

4278.67333736 9.74884202318
4134.20916265 14.8038215541
1.03494360566 0.00439255803019


In [335]:
2*2

4

### Look at neutrality data

In [925]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 1
decimate = 10
def get_amp(i):
    
    fname = r'/data/old_trap/20211001/bead1/neutrality2/X_100Vpp_139Hz_spin_xz_100Vpp_77kHz_delay_with_height_fb_redo/X_100Vpp_'+str(i)+'.h5'

    drive_freq = 139
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    
    freq2 = np.fft.rfftfreq(len(bb.other_data[3]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[3]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)

#     fit_kwargs = {'A': 0.23, 'f': drive_freq, 'phi': 0, 
#                   'error_A': 1, 'error_f': 10, 'error_phi': 0.5, 'errordef': 1,
#                   'limit_A': [0, 1], 'fix_f': True, 'fix_phi': False,
#                   'print_level': 0}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
#     xx2 = signal.filtfilt(b, a, bb.other_data[3])[::decimate]    
#     m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
#     phi_tmp = m1_tmp.values[2]

#     print(phi_tmp, phi_tmp2, m1_tmp.values[0]*100, np.std(xx2)*np.sqrt(2)*100)
    
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 'sigma': 1100,
                  'error_A': 0.1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':50,
                  'limit_A': [-100000, 100000], 'limit_sigma':[100,30000], 'fix_phi': True, 'fix_f': True,
                  'print_level': 0}

    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    drive_freq = drive_freq*2
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2+np.pi/2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_A': [-100000, 100000], 'fix_f': True, 'fix_phi': True,
                  'print_level': 0}

        
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]
    m2_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)

    #     print(m1_tmp.values)
    return m1_tmp.values[0], m2_tmp.values[0], m1_tmp.values[3], m1_tmp.fval

In [923]:
i=0
get_amp(i)[0]

5.088916634093039

In [879]:
i = 2
fname = r'/data/old_trap/20211001/bead1/neutrality4/test_alternate/test_voltages/Y_50Vpp_'+str(i)+'.h5'
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
print("voltage: ", voltage, np.mean(bb.other_data[3])*100, np.mean(bb.other_data[4])*100)
drive_freq = 139
bandwidth=2
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[5])
voltage_100_1 = np.std(xx2)*np.sqrt(2)*100
xx3 = signal.filtfilt(b, a, bb.other_data[6])
voltage_100_2 = np.std(xx3)*np.sqrt(2)*100
print("voltage: ", voltage_100_1, voltage_100_2)

voltage:  50 -0.540029755093 -0.445538261997
voltage:  24.0272414237 0.0888447340811


In [890]:
voltage, voltage_100

(50, 96.209444386695921)

In [184]:
# amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay = []
# for i in range(1000):
#     print(i,'/500')
#     try:
#         amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay.append(get_amp(i))
#     except:
#         print("******")
# amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay = np.array(amps_no_fb_spin_xz_200Vpp_x_50Vpp_delay)

In [891]:
def get_amp_wraper(i):
    try:
        return get_amp(i)
    except:
        return 0,0,0,0

In [926]:
amps_with_fb_spin_xz_100Vpp_77Hz_x_100Vpp_delay_redo = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraper)(i) for i in tqdm(range((2000)))))

100%|██████████| 2000/2000 [00:34<00:00, 55.62it/s]


In [248]:
# fvals = amps_no_fb_spin_xz_200Vpp_x_100Vpp_delay[:,2]
# thres = np.mean(fvals)+2*np.std(fvals)
# thres

In [860]:
i=0; j=2
_,ax = plt.subplots()
dat = amps_with_fb_spin_xz_100Vpp_77kHz_y_minus_100Vpp_delay
ax.scatter(dat[:,i], dat[:,j])

# dat2 = amps_with_fb_spin_xz_100Vpp_77kHz_x_minus_100Vpp_delay
# ax.scatter(dat2[:,i], dat2[:,j])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
# dat = amps_no_fb_spin_xz_100Vpp_77kHz_x_100Vpp_delay_height_fb
# thresh = 1600
# amps_no_fb_spin_xz_100Vpp_77kHz_x_100Vpp_delay_height_fb_2 = np.array([aa for aa,bb in zip(dat,dat[:,2]) if (bb<thresh)])

In [375]:
voltage, voltage100

(23.353406758381592, 46.690643728039987)

In [927]:
j = 0
data = amps_with_fb_spin_xz_100Vpp_77Hz_x_100Vpp_delay[:,j]/scale*voltage/voltage_100
data2 = amps_with_fb_spin_xz_100Vpp_77Hz_x_100Vpp_delay_redo[:,j]/scale*voltage/voltage_100
# data3 = amps_with_fb_spin_xz_100Vpp_77kHz_x_150Vpp_delay[:,j]/scale*voltage/voltage150

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data, label='150Vpp-x, xz-spin-100Vpp-77kHz')
ax.scatter(range(len(data2)), data2, label='150Vpp-y, xz-spin-100Vpp-77kHz', alpha=0.6)
# ax.scatter(range(len(data3)), data3, label='150Vpp-x, xz-spin-100Vpp-77kHz', alpha=0.6)

# ax.scatter(range(len(data4)), data4, label='20V-x, xy-spin-30V, w/o height fb')
ax.plot(np.arange(0,1000), np.arange(0,1000)*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean, std: ', np.mean(data), np.std(data)/np.sqrt(len(data)))
print('mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))
# print('mean, std: ', np.mean(data3), np.std(data3)/np.sqrt(len(data3)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean, std:  -0.000655320614665 0.000571212167888
mean, std:  9.23990935359e-05 0.000571698344227


In [928]:
data_all_100_2 = np.array(list(data)+list(data2))#+list(data3))
np.mean(data_all_100_2), np.std(data_all_100_2)/np.sqrt(len(data_all_100_2)), len(data_all_100_2)

(-0.00028146076056470717, 0.00040412315843237897, 4000)

In [933]:
data_all_100_all = np.array(list(data_all_100)+list(data_all_100_2))#+list(data3))


In [934]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

n_bins = 21
range_pram = 0.1
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data_all_100_all, bins=n_bins, range=(-range_pram, range_pram), label='100Vpp, 49kHz')
# entries2, bins, patches = ax.hist(data2, bins=n_bins, range=(-range_pram, range_pram), label='150Vpp, 77kHz, after weekend', alpha=0.6)
# entries3, bins, patches = ax.hist(data3, bins=n_bins, range=(-range_pram, range_pram), label='150Vpp, 77kHz', alpha=0.6)
ax.set(xlabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [935]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

titles = ['', '', '']

for i,ee in enumerate([entries]):
    x = bins_fit
    y= np.array(ee)
    sigma_bin = np.sqrt(ee)
    
    n = len(x)                          #the number of data
    mean = 0                  #note this correction
    sigma = 0.023       #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print(titles[i], ": mean: ", popt[1], " ,std (on the mean): ", np.sqrt(pcov)[1,1], " ,sigma: ", popt[2]/np.sqrt(1000))

 : mean:  -0.000615497819225  ,std (on the mean):  0.000131442301244  ,sigma:  -0.000815901459391


In [119]:
datasets = [amps_wo_fb_spin_zx_75V_x_20V,amps_wo_fb_spin_zx_100V_x_25V, amps_wo_fb_spin_zx_150V_x_25V]
voltages = [voltage20, voltage25, voltage25]
spin_voltage = [75,100, 150]
means = []
errors = []
for dat,vv in zip(datasets, voltages):
    means.append(np.mean(dat[:,0])/scale*voltage/vv)
    errors.append(np.std(dat[:,0])/scale*voltage/vv/np.sqrt(len(dat[:,0])))

_,ax = plt.subplots()
ax.errorbar(spin_voltage, means, yerr=errors, fmt='o', label='zx plane spin')
ax.set(xlabel='Spin voltage', ylabel=r'$\epsilon$', xlim=(20,160), ylim=(-0.001,0.01))

datasets = [amps_wo_fb_spin_xy_30V_x_20V,amps_wo_fb_spin_xy_75V_x_20V, amps_wo_fb_spin_xy_100V_x_20V]
voltages = [voltage20, voltage20, voltage20]
spin_voltage = [30,75, 100]
means2 = []
errors2 = []
for dat,vv in zip(datasets, voltages):
    means2.append(np.mean(dat[:,0])/scale*voltage/vv)
    errors2.append(np.std(dat[:,0])/scale*voltage/vv/np.sqrt(len(dat[:,0])))

ax.errorbar(spin_voltage, means, yerr=errors, fmt='o', label='xy plane spin')
ax.plot(np.arange(20,160), np.arange(20,160)*0, '--r')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [384]:
j = 0
data = amps_with_fb_spin_xy_100V_x_20V_delay_2[:,j]/scale*voltage/voltage20_1
data2 = amps_wo_fb_spin_zx_100V_x_25V[:,j]/scale*voltage/voltage25

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(data)), data, label='20V-x, xy-spin-100V, w/ delay, w/ height fb')
ax.scatter(range(len(data2)), data2, label='25V-x, xy-spin-100V, w/ delay, w/o height fb')

ax.plot(np.arange(0,3000), np.arange(0,3000)*0, '--k')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean, std: ',np.mean(data), np.std(data)/np.sqrt(len(data)))
print('mean, std: ', np.mean(data2), np.std(data2)/np.sqrt(len(data2)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean, std:  0.00543937010689 0.000832138675641
mean, std:  0.00364790172652 0.000627235885947


In [181]:
datasets = [amps_wo_fb_spin_xy_100V_x_20V2[1200:],amps_wo_fb_spin_xy_100V_x_50V_delay2, amps_wo_fb_spin_xy_100V_x_100V_delay2]
spin_voltage = [20,50, 100]
means = []
errors = []
for dat,vv in zip(datasets, voltages):
    means.append(np.mean(dat))
    errors.append(np.std(dat)/np.sqrt(len(dat)))

_,ax = plt.subplots()
ax.errorbar(spin_voltage, means, yerr=errors, fmt='o', label='xy plane spin, 100V')
ax.plot(np.arange(15,110), np.arange(15,110)*0, '--r')
ax.set(xlabel='Oscillating voltage', ylabel=r'$\epsilon$', xlim=(15,110), ylim=(-0.001,0.004))
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
def get_dcV(i):
    
    fname = r'/data/old_trap/20210715/bead2/neutrality/no_height_fb_electrodes_x_20V_41_Hz_spin_xy_30V_50kHz_files_3000/turbombar_powfb_xyzcool_'+str(i)+'.h5'
    bb = bu.DataFile();
    bb.load(fname);
    bb.load_other_data()
    return np.mean(bb.other_data[:,5000:85000], axis=1)*100

In [146]:
get_dcV(0)

array([-3.79819842, -0.56254724, -0.57267787,  0.66225783,  0.92968812,
       -0.39438807, -0.0508512 ,  0.22620381])

In [144]:
voltages_elec3_30V_spin = np.array(Parallel(n_jobs=40)(delayed(get_dcV)(i) for i in tqdm(range(3000))))


100%|██████████| 3000/3000 [00:09<00:00, 314.37it/s]

In [313]:
i=1
fname = r'/data/old_trap/20210817/bead1/neutrality/height_fb_0_7_x_41Hz_50V_delay_10s__spin_xz_100V_50kHz_files_3000/x_neutrality_50V_41Hz_'+str(i)+'.h5'
    
drive_freq = 41
bb = bu.DataFile();
bb.load(fname);
bb.load_other_data()
    
b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
xx2 = signal.filtfilt(b, a, bb.other_data[3])[::decimate]
m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
xx2 = signal.filtfilt(b, a, bb.pos_data[0])[::decimate]

_,ax = plt.subplots()
ax.scatter(range(50000), bb.other_data[3])

NameError: name 'fit_kwargs' is not defined